In [5]:
import numpy as np
import traceback
import concurrent.futures
import datetime
from matplotlib import pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from browsermobproxy import Client
from selenium import webdriver
from typing import List

from util import is_json, parse_har, write_har, get_driver, get_proxy
from crawler import process_event
from feature_gen import gen_feature
from gen_types import Event, System

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def worker(sys: System, drivers: List[webdriver.Firefox], proxy: Client, worker_idx: int):
    while len(sys.mq) != 0:
        curr_event = sys.pop_event()
        # print("Processing event {}".format(curr_event))
        process_event(drivers[worker_idx], proxy, curr_event, sys)

In [7]:
# ---------------------
num_worker = 36
num_user = 500
mission_hour = 4
# ---------------------

mission_minutes = 5*60
mission_seconds = 5*60*60

In [8]:
# random_walk_page(driver, "https://cs.uchicago.edu", 5)
# har = proxy.har # returns a HAR JSON blob
proxy, server = get_proxy(8080)
pool = ThreadPoolExecutor(num_worker)

drivers = []
for i in range(0, num_worker):
    drivers.append(get_driver(proxy))
    
try:
    sys = System(mission_minutes, "https://cs.uchicago.edu")
    sys.init_mq(num_user)

    futures = []
    for worker_idx in range(num_worker):
        futures.append(pool.submit(worker, sys, drivers, proxy, worker_idx))

    for fut in futures:
        fut.result()
except:
    traceback.print_exc()
finally:
    for i in range(0, num_worker):
        drivers[i].quit()
    server.stop()

The geckodriver version (0.31.0) detected in PATH at /usr/local/bin/geckodriver might not be compatible with the detected firefox version (119.0); currently, geckodriver 0.33.0 is recommended for firefox 119.*, so it is advised to delete the driver in PATH and retry
The geckodriver version (0.31.0) detected in PATH at /usr/local/bin/geckodriver might not be compatible with the detected firefox version (119.0); currently, geckodriver 0.33.0 is recommended for firefox 119.*, so it is advised to delete the driver in PATH and retry
The geckodriver version (0.31.0) detected in PATH at /usr/local/bin/geckodriver might not be compatible with the detected firefox version (119.0); currently, geckodriver 0.33.0 is recommended for firefox 119.*, so it is advised to delete the driver in PATH and retry
The geckodriver version (0.31.0) detected in PATH at /usr/local/bin/geckodriver might not be compatible with the detected firefox version (119.0); currently, geckodriver 0.33.0 is recommended for fir

In [ ]:
aggregate_har = []
for hars in sys.har_buffer:
    aggregate_har += hars

In [ ]:
test_pdf = parse_har(aggregate_har)
test_pdf.sort_values(by=['start_time'])
test_pdf.head(20)

In [ ]:
test_pdf['relative'] = test_pdf.start_time.astype(np.int64) // 10 ** 9
test_pdf['relative'] = test_pdf['relative'] - min(test_pdf['relative'])
test_pdf.head(200)

In [ ]:
load_path = "../data/cs.uchicago.edu/"
fpd = gen_feature(load_path)
fpd.head(5)

In [ ]:
# Plot access pattern
plot_url = 'https://cs.uchicago.edu/'
plot_pdf = test_pdf[test_pdf['url'] == plot_url]
plot_pdf.head(50)